# Detecção de Hate Speech e Análise de Sentimentos de Comentários Transmissões de Streamers do Jogo League of Legends

##### Dependências

Estes são os requisitos necessários para seguir este tutorial:

- [Python 3.x](https://www.python.org/downloads/)
- [Jupyter Notebook](http://jupyter.org/install)
- [Pandas](https://pandas.pydata.org)
- [NLTK](https://www.nltk.org/)
- [Polyglot](http://polyglot.readthedocs.io/en/latest/Installation.html)
- [Scikit-Learn](http://scikit-learn.org/stable/install.html)
- [Scipy](https://www.scipy.org/install.html)
- [Wordcloud](https://github.com/amueller/word_cloud)
- [Matplotlib](https://matplotlib.org/users/installing.html)
- [Hatesonar](https://github.com/Hironsan/HateSonar)


## Leitura dos dados


In [ ]:
import pandas as pd
#Caminhos para os arquivos

#filename = "Dataset/C9Sneaky_comments.csv"
#filename = "Dataset/DoubleLift_comments.csv"
#filename = "Dataset/DoubleLift_comments.csv"
#filename = "Dataset/lolTyler1_comments.csv"
#filename = "Dataset/Nightblue3_comments.csv"
#filename = "Dataset/Nightblue3_comments.csv"
#filename = "Dataset/TFBlade_comments.csv"
#filename = "Dataset/Trick2g_comments.csv"
filename = "Dataset/TSM_Bjergsen_comments.csv"
#filename = "Dataset/Yassuo_comments.csv"
#filename = "Dataset/TwitchDataset.csv" 
# Função que faz a leitura do dataset.
def read_data(path, sep):
    data = pd.read_csv(path, sep=sep)
    return data

In [ ]:
dataset = read_data(filename, sep="\t")
print(dataset.comment.head(5))

### Número de Visualizações por Canal

In [ ]:
import pandas as pd

def ch_num_of_views(text):
    now = []
    strm = []
    for data in text:
        df_prof=pd.read_csv(data, sep='\t')
        df1=list(df_prof.iloc[0:1].user_name)[0]
        df2=list(df_prof.iloc[0:1].user_vcount)[0]
        df2=str(df2)
        now.append(df2)
        strm.append(df1)
    return strm,now

In [ ]:
#Caminhos para os arquivos

#file_pro= = "Profiles/C9Sneaky.csv"
#file_pro = "Profiles/DoubleLift.csv"
#file_pro = "Profiles/foggedftw2.csv"
#file_pro = "Profiles/lolTyler1.csv"
#file_pro = "Profiles/Nightblue3.csv"
#file_pro = "Profiles/Rush.csv"
#file_pro = "Profiles/TFBlade.csv"
#file_pro = "Profiles/Trick2g.csv"
#file_pro = "Profiles/Yassuo.csv"
#file_pro = "Profiles/TSM_Bjergsen.csv"
file_pro = ["Profiles/TSM_Bjergsen.csv","Profiles/Yassuo.csv","Profiles/Trick2g.csv","Profiles/TFBlade.csv","Profiles/Rush.csv",
            "Profiles/Nightblue3.csv","Profiles/lolTyler1.csv","Profiles/foggedftw2.csv",
            "Profiles/DoubleLift.csv","Profiles/C9Sneaky.csv"]
           
strm_list,now_list=ch_num_of_views(file_pro)
dict_now_strm= {'Streamer':strm_list,'Número de Visualizações':now_list}
df_now = pd.DataFrame(dict_now_strm,columns=['Streamer','Número de Visualizações'])
df_now['Número de Visualizações']= df_now['Número de Visualizações'].astype('int')
df_now=df_now.sort_values(by='Número de Visualizações',ascending=False)
df_now.to_csv(r'Num_visualizações/Num_views_ch.csv', index = None, header=True,sep='\t')

In [ ]:
import pandas as pd
file = "Profiles/TSM_Bjergsen.csv"
df = pd.read_csv(file, sep='\t')

In [ ]:
df

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

file_sep=w= 'Num_visualizações/Num_views_ch.csv'
df_view=pd.read_csv(file_view,sep='\t')
plt.rcParams.update({'font.size':15})
# Plotando gráfico de classificação por streamer
ax=df_view[['Número de Visualizações']].plot(figsize=(10,5),kind='bar')
ax.set_xticklabels(df_view['Streamer'], rotation=45)
plt.legend(loc='best')
plt.xlabel('Streamers')
plt.ylabel('Número de Visualizações')
plt.title('Número de Visualizações por Canal')

In [ ]:
df_view

## Pré-processamento

### 1 - Letras Minúsculas

In [ ]:
# Função que converte o texto em letras minúsculas.
def lower_case(text):
    return text.lower()

### 2 - Removendo Acentuação

In [ ]:
# Para realizar essa tarefa é necessário importar a biblioteca "unicodedata".
import unicodedata
# Função que remove a acentuação do texto, onde 'NFKD' é o codigo da forma normal para entrada de strings unicode.
def remove_accentuation(text):
    text = unicodedata.normalize('NFKD', str(text)).encode('ASCII','ignore')
    return text.decode("utf-8")

### 3 - Tokenização

In [ ]:
# Função que faz a tokenização do texto.
def tokenize(text):
    return text.split(" ")

### 4 - Remoção de Ruídos

In [ ]:
# Função que remove pontuação e substitui alguns termos e caracteres do texto.
def remove_noise(text):
    # re.sub(replace_expression, replace_string, target)
    new_text = re.sub(r"\.|,|;|!|\?|\"|\:|\+|\-|\*|\#|\'|\~",'', text)
    new_text = re.sub(r"@\S+", "MENTION",new_text)
    new_text = re.sub(r"\<3+", "LOVE", new_text)
    new_text = re.sub(r"\bf\b", "FLASH", new_text)
    new_text = re.sub(r"[A-Z0-9]+|\.|\!|\,",'', new_text)
    new_text = re.sub(r"\$|\@|\(|\)|\&|\¨|\_|\=",'', new_text)
    new_text = re.sub(r"\\|\||\/|\>|\<|\[|\]|\{|\}",'', new_text)
    new_text = re.sub(r"\n|\t|\r",'', new_text)
    new_text = re.sub(r"\`|\´|\^|\%|\;|\:|\§|\ª|\º|\₢|\°",'', new_text)
    
    return new_text

### 5 - Remoção de Números

In [ ]:
# Função para remover números do texto
def remove_numbers(text):
    # re.sub(replace_expression, replace_string, target)
    new_text = re.sub(r"[0-9]+", "", text)
    return new_text

### 6 - Remoção de Stopwords

In [ ]:
import nltk

from nltk.corpus import stopwords
nltk_stop = set(stopwords.words('english'))# indica o conjunto de stopwords específico do idioma inglês.
# Função para remover stop words do texto.
def remove_stop_words(text, stopWords):
    for sw in stopWords:
        text = re.sub(r'\b%s\b' % sw, "", text)
    return text

### 7 - Redução de Comprimento

In [ ]:
#Padrão de comprimento de palavras utilizado
lengthening_pattern = "a{3,}|b{3,}|c{3,}|d{3,}|e{3,}|f{3,}|g{3,}|h{3,}|i{3,}|j{3,}|" \
                        "k{3,}|l{3,}|m{3,}|n{3,}|o{3,}|p{3,}|q{3,}|r{3,}|s{3,}|t{3,}|" \
                        "u{3,}|v{3,}|x{3,}|w{3,}|y{3,}|z{3,}"
# Função usada para reduzir o comprimento de palavras no texto. Exemplo: 'looove' ----> 'love'.
def lengthening_reduction(text, lenPattern):
    lengthenings = re.findall(lenPattern, text)
    if lengthenings:
        lengthenings = lengthenings[0]
        text = re.sub(lengthenings, lengthenings[0:1], text)
    
    return text

### 8 - Stemming

In [ ]:
from nltk.stem.porter import *
# Instanciando o Porter stemmer
stemmer = PorterStemmer()

# Funcão para Stemming
def stemming(token, stemmer):
    return stemmer.stem(token)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
# Instanciando o Snowball stemmer
stemmer_en = SnowballStemmer("english")

### 9 - Lemmatizing

In [ ]:
import nltk
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk

# The argument "special_terms" refers to terms that should not be parsed.
def lemmatizing(text, special_terms=[]):
    tokenPattern = r"[A-Z0-9-']+|\.|\!|,"
    tokens = tokenize_refined(text, tokenPattern)
    
    tags = nltk.pos_tag(tokens)
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []
    for token, tag in tags:
        if token in special_terms:
            lemmatized_tokens.append(token)
        else:
            lemma = lemmatizer.lemmatize(token, pos=get_wordnet_pos(tag))
            lemmatized_tokens.append(lemma)

    lemmatized_tokens_string = " ".join([token for token in lemmatized_tokens])
    return lemmatized_tokens_string

### 10 - Removendo Online Social Networking


In [ ]:
import re
def removalOSN(text):
    text = re.sub("#\\S+", "", text)
    text = re.sub("@\\S+", "", text)
    return text

In [ ]:
#Função para substituir expressões próprias do Twitch
#def subTwitchslangs(text):
#text = re.sub("\bpog\b", "surprised", text)
    #text = re.sub("\bpogu\b", "surprised", text)
    #text = re.sub("\bpogger\b", "surprised", text)
    #text = re.sub("\bpogchamp\b", "surprised", text)
    #text = re.sub("\btrihard\b", "excited", text)
    #text = re.sub("\bkappa\b", "sarcasm", text)
    #text = re.sub("\bminik\b", "sarcasm", text)
    #text = re.sub("\bkappahd\b", "sarcasm", text)
    #text = re.sub("\bkappaross\b", "sarcasm", text)
    #text = re.sub("\bkappaclaus\b", "sarcasm", text)
    #text = re.sub("\bkappapride\b", "sarcasm", text)
    #text = re.sub("\bkeppo\b", "sarcasm", text)
    #text = re.sub("\bkappa\b", "sarcasm", text)
    #text = re.sub("\blul\b", "sarcasm", text)
    #text = re.sub("\bkekw\b", "laugh", text)
    #text = re.sub("\blmao\b", "laugh", text)
    #text = re.sub("\bpepehands\b", "sad", text)
    #return text
    

In [ ]:
#Função para substituir OSN
def replacementOSN(text):
    text = re.sub("#\\S+", "HASHTAG", text)
    
    text = re.sub("@\\S+", "MENTION", text)
    
    return text

In [ ]:
#função para interpretação de hashtags
def interpretationHashtags(text):
    hashtags = re.findall("#\\S+", text)
    for hashtag in hashtags:
        words = re.findall('[a-zA-Z][^A-Z]*', hashtag)
        wordsAsText = " ".join([word for word in words])
        text = re.sub(hashtag, wordsAsText, text)
    
    return text

In [ ]:
#função para remover urls
def removeURL(text):
    text = re.sub("http\\S+\\s*", "", text)
    
    return text

## Transformação dos dados

Nesta etapa serão chamadas as funções criados anteriormente para tratar os dados.

In [ ]:
#Caminhos para os arquivos

#filename = "Dataset/C9Sneaky_comments.csv"
#filename = "Dataset/DoubleLift_comments.csv"
#filename = "Dataset/DoubleLift_comments.csv"
#filename = "Dataset/lolTyler1_comments.csv"
#filename = "Dataset/Nightblue3_comments.csv"
#filename = "Dataset/Nightblue3_comments.csv"
#filename = "Dataset/TFBlade_comments.csv"
#filename = "Dataset/Trick2g_comments.csv"
filename = "Dataset/TSM_Bjergsen_comments.csv"
#filename = "Dataset/Yassuo_comments.csv"
#filename = "Dataset/TwitchDataset.csv" 
dataset = read_data(filename, sep="\t")

In [ ]:
print(dataset.comment.head(5))

### Quantidade de comentários brutos

In [ ]:
print(len(dataset))

In [ ]:
#Função que faz todo o preprocessamento, aplicando as funções criadas acima.
def preprocessing(dataframe, fieldName, config):   
    
    if config["remove_numbers"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(remove_numbers)
        
    if config["lowercase"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(lower_case)
    
    if config["replace_hashtagsMentions"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(replacementOSN)
        
    if config["remove_accentuation"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(remove_accentuation)
    
    if config["remove_stopwords"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(remove_stop_words, stopWords=nltk_stop)
    
    if config["reduce_lengthening"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(lengthening_reduction, lenPattern=lengthening_pattern)
    
    if config["stemming"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(stemming, stemmer=stemmer)
    
    if config["lemmatizing"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(lemmatizing)
    
    if config["remove_hashtagsMentions"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(removalOSN)
          
    if config["interpretHashtags"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(interpretationHashtags)
    
    if config["remove_urls"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(removeURL)
        
    if config["remove_noise"] == True:
        dataframe[fieldName] = dataframe[fieldName].apply(remove_noise)
    
    return dataframe

In [ ]:
config = {
            "remove_numbers": True, \
            "lowercase": True, \
            "remove_accentuation": True, \
            "remove_stopwords": True, \
            "reduce_lengthening": True, \
            "stemming": False, \
            "lemmatizing": False, \
            "remove_hashtagsMentions": True, \
            "replace_hashtagsMentions": True, \
            "interpretHashtags": True, \
            "remove_urls": True, \
            "remove_noise": True}

In [ ]:
dataset.dropna(inplace = True)# remove linhas com valores NAN
dataset.drop_duplicates(inplace = True)# remove elementos duplicados
dataset_pproc = preprocessing(dataset, fieldName="comment", config=config)
dataset_pproc.comment.head(5)

### Quantidade de comentários após preprocessamento

In [ ]:
print(len(dataset_pproc))

### Salvando os comentários preprocessados em um arquivo .csv

In [ ]:
#df = pd.DataFrame(dataset_pproc,columns=['comment'])
#df.to_csv (r'PreprocessDataset/TwitchDataset_pproc_Dataset.csv', index = None, header=True,sep='\t')

# Visualização em WordCloud

In [ ]:
%matplotlib inline
from os import path
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

# Wordcloud para comentários
text = ' '.join(dataset_pproc.comment.values)# retira uma lista com os valores contidos no dataset
words = text.split()
text = (" ".join(sorted(set(words),key=words.index)))#organiza e seleciona apenas palavras unicas da lista
text = re.sub(r'[^A-Za-z0-9 ]+','', text)# substitui possíveis ruídos no texto

#print(text)
# Gerando a imagem wordcloud
wordcloud = WordCloud().generate(text)
wordcloud = WordCloud(max_font_size=55,background_color="white", max_words=50).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Modelagem de Tópicos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Função para vetorização do dataset
def transformData(data, fieldName, my_tokenizer, weight):
    
    if weight == "TP":
        vectorizer = CountVectorizer(tokenizer=my_tokenizer, binary=True)
        X = vectorizer.fit_transform(data[fieldName])
    
    elif weight == "TF":
        vectorizer = CountVectorizer(tokenizer=my_tokenizer)
        X = vectorizer.fit_transform(data[fieldName])
        
    elif weight == "TFIDF":
        vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
        X = vectorizer.fit_transform(data[fieldName])

    return (vectorizer, X)

In [ ]:
# Chamando função para vetorizar o dataset
vectorizer, X = transformData(dataset_pproc, fieldName="comment", my_tokenizer=tokenize, weight="TFIDF")

In [ ]:
#print(vectorizer.get_feature_names())

In [ ]:
#variável contento as caracteristicas extraídas do dataset
tfidf_feature_names= vectorizer.get_feature_names()

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

#Função para mostrar os principais tópicos
def display_topics(model, feature_names, top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Tópico %d: " % (topic_idx) +
                " ".join([feature_names[i]
                        for i in topic.argsort()[:-top_words -2:-1]]))

topics = 5
top_words = 5
# Exexutando os modelos LDA e LSA 
lda = LatentDirichletAllocation(n_components=topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(X)
lsa = TruncatedSVD(n_components=topics, algorithm='randomized', n_iter=5, random_state=1).fit(X)

#Mostrando os resultados para cada modelo
print('Modelo LDA')
display_topics(lda, tfidf_feature_names, top_words)
print('Modelo LSA')
display_topics(lsa, tfidf_feature_names, top_words)


# Análise de Sentimentos

In [ ]:
from polyglot.text import Text
import matplotlib.pyplot as plt


#print("{:<16}{}".format("Word", "Polarity")+"\n"+"-"*30)

def sent_analize_pos(text):
    Positivos = []
    for w in text.words:
        #print(w.polarity)
        if (w.polarity == 1):
            Positivos.append("{:<16}{:>2}".format(w, w.polarity))
    return Positivos

def sent_analize_neg(text):
    Negativos = []
    for w in text.words:
        #print(w.polarity)
        if (w.polarity == -1):
            Negativos.append("{:<16}{:>2}".format(w, w.polarity))
    return Negativos

def sent_analize_neutros(text):
    Neutros = []
    for w in text.words:
        #print(w.polarity)
        if (w.polarity == 0):
            Neutros.append("{:<16}{:>2}".format(w, w.polarity))
    return Neutros
text = Text(' '.join(dataset_pproc.comment.values))
Positivos = sent_analize_pos(text)
Negativos = sent_analize_neg(text)
Neutros = sent_analize_neutros(text)
Num_pos= len(Positivos)
Num_neg= len(Negativos) 
Num_neutros= len(Neutros)

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Positivos', 'Negativos', 'Neutros', 
sizes = [Num_pos, Num_neg, Num_neutros]
explode = (0.1, 0.1, 0.1)  # Explode all sides
plt.rcParams.update({'font.size':15})
fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=200)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Análise de Sentimentos dos comentários')

plt.show()


print(" Positivos: %d\n" % Num_pos,
      "Negativos: %d \n" % Num_neg,
      "Neutros: %d" % Num_neutros)

# Detecção de HateSpeech

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
from hatesonar import Sonar

# Função que utiliza o hatesonar para classificar os comentãrios linha por linha em hate speech, offensive language e neither
# A função salva os resultados em um arquivo .csv
def detect_hs(text):
    warnings.filterwarnings(action='ignore')
    sonar = Sonar()
    with open('HS_detection/TwitchDataset_HS_detect_saved.csv','w',encoding='utf8') as f:
        titles = f"comment\tTop_Class\tHateSpeech_Confidence\tOfensive_Language_Confidence\tNeither_Confidence\n"
        f.write(titles)
        temp_list = []
        for dado in text.itertuples():
            dado = dado.comment
            dit = sonar.ping(dado)# método que faz a classificação: o resultado é um dicionário contendo os resultad
            texto = dit['text']
            classif = dit['top_class']
            classes = dit['classes']
            hs_confidence = classes[0]['confidence']
            #hs_class = classes[0]['class_name']
            ol_confidence = classes[1]['confidence']
            #ol_class = classes[1]['class_name']
            neither_confidence = classes[2]['confidence']
            #neither_class = classes[2]['class_name']
            temp_list.append(texto + classif + str(hs_confidence)  + str(ol_confidence) + str(neither_confidence))

            f.write(f"{texto}\t{classif}\t{hs_confidence}\t{ol_confidence}\t{neither_confidence}\n")
    f.close()

In [ ]:
# Executando a função de detecção de hate speech
file = 'PreprocessDataset/TwitchDataset_pproc_Dataset.csv'# caminho para o arquivo preprocessado desejado
df = pd.read_csv(file,sep='\t')
df=df.dropna(subset=['comment'])
df =detect_hs(df)

### Visualização dos Resultados da Detecção de Hate Speech por Streamer

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

# Lista com os caminhos para os arquivos de detecção de hate speech de cada streamer
file = ['HS_detection/C9sneaky_HS_detect_saved.csv','HS_detection/DoubleLift_HS_detect_saved.csv','HS_detection/foggedftw2_HS_detect_saved.csv',
        'HS_detection/lolTyler1_HS_detect_saved.csv','HS_detection/Nightblue3_HS_detect_saved.csv','HS_detection/Rush_HS_detect_saved.csv',
        'HS_detection/TFBlade_HS_detect_saved.csv','HS_detection/Trick2g_HS_detect_saved.csv','HS_detection/TSM_Bjergsen_HS_detect_saved.csv',
        'HS_detection/Yassuo_HS_detect_saved.csv','HS_detection/TwitchDataset_HS_detect_saved.csv']




# Função para contagem das classificações por streamer e de todo o dataset

def hs_for_stmrs(text):
    hate_speech = []
    of_lang = []
    neither= []
    total = 0
    for dado in text.itertuples():
        dados = dado.Top_Class
        commt= dado.comment
        if (dados == 'hate_speech'):
            hate_speech.append(dados+':'+commt)
            num_hs = len(hate_speech)
           
        elif(dados == 'offensive_language'):
            of_lang.append(dados+':'+commt)
            num_of_lg = len(of_lang)

        elif(dados == 'neither'):
            neither.append(dados+':'+commt)
            num_nt = len(neither)
           
    total = (num_hs+num_of_lg+num_nt)
    return num_hs,num_of_lg,num_nt,total

# Iteração sobre a lista de caminhos de arquivos
for i in file:
    if (i== 'HS_detection/C9sneaky_HS_detect_saved.csv'):
        strm1  = 'C9sneaky'
        df2= pd.read_csv(i,sep='\t')
        hs1,ol1,nt1,tot1 = hs_for_stmrs(df2)
    
    elif (i== 'HS_detection/DoubleLift_HS_detect_saved.csv'):
        strm2  = 'DoubleLift'
        df2 = pd.read_csv(i,sep='\t')
        hs2,ol2,nt2,tot2 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/foggedftw2_HS_detect_saved.csv'):
        strm3  = 'foggedftw2'
        df2 = pd.read_csv(i,sep='\t')
        hs3,ol3,nt3,tot3 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/lolTyler1_HS_detect_saved.csv'):
        strm4  = 'lolTyler1'
        df2 = pd.read_csv(i,sep='\t')
        hs4,ol4,nt4,tot4 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/Nightblue3_HS_detect_saved.csv'):
        strm5  = 'Nightblue3'
        df2 = pd.read_csv(i,sep='\t')
        hs5,ol5,nt5,tot5 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/Rush_HS_detect_saved.csv'):
        strm6  = 'Rush'
        df2 = pd.read_csv(i,sep='\t')
        hs6,ol6,nt6,tot6 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/TFBlade_HS_detect_saved.csv'):
        strm7   = 'TFBlade'
        df2 = pd.read_csv(i,sep='\t')
        hs7,ol7,nt7,tot7 = hs_for_stmrs(df2)
            
    elif (i== 'HS_detection/Trick2g_HS_detect_saved.csv'):
        strm8 = 'Trick2g'
        df2 = pd.read_csv(i,sep='\t')
        hs8,ol8,nt8,tot8 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/TSM_Bjergsen_HS_detect_saved.csv'):
        strm9  = 'TSM_Bjergsen'
        df2 = pd.read_csv(i,sep='\t')
        hs9,ol9,nt9,tot9 = hs_for_stmrs(df2)

    elif (i== 'HS_detection/Yassuo_HS_detect_saved.csv'):
        strm10 = 'Yassuo'
        df2 = pd.read_csv(i,sep='\t')
        hs10,ol10,nt10,tot10 = hs_for_stmrs(df2)
    
    elif (i== 'HS_detection/TwitchDataset_HS_detect_saved.csv'):# caminho para todo o dataset com todos os comentários
        strm11 = 'TwitchDataset_All Data'
        df2 = pd.read_csv(i,sep='\t')
        hs11,ol11,nt11,tot11 = hs_for_stmrs(df2)

In [ ]:
# Unindo os resultados em um lista para transformá-los em um dataframe pandas
qt_hs=[hs1,hs2,hs3,hs4,hs5,hs6,hs7,hs8,hs9,hs10]
strms= [strm1,strm2,strm3,strm4,strm5,strm6,strm7,strm8,strm9,strm10]
qt_ol =[ol1,ol2,ol3,ol4,ol5,ol6,ol7,ol8,ol9,ol10] 
qt_nt =[nt1,nt2,nt3,nt4,nt5,nt6,nt7,nt8,nt9,nt10]
tot =[tot1,tot2,tot3,tot4,tot5,tot6,tot7,tot8,tot9,tot10]
dict_Hs = {'Streamer':strms,'Hate Speech':qt_hs,'Offensive Language':qt_ol, 'Neither':qt_nt,'Total':tot}
dict_Alldata = {'All Twitch Dataset':[strm11],'Hate Speech':[hs11],'Offensive Language':[ol11], 'Neither':[nt11],'Total':[tot11]}
# Transformando os resultados em um dataframe pandas
df12 = pd.DataFrame(dict_Hs,columns=['Streamer','Hate Speech','Offensive Language','Neither','Total'])
df13 = pd.DataFrame(dict_Alldata,columns=['All Twitch Dataset','Hate Speech','Offensive Language','Neither','Total'])
# Salvando resultados em um arquivo .csv
#df12.to_csv (r'hs_strmrs.csv', index = None, header=True,sep='\t',)
#df13.to_csv (r'hs_alldataset.csv', index = None, header=True,sep='\t',)


In [ ]:
df12

In [ ]:
df13

### Visualizando em um Gráfico de Barras

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size':15})
# Plotando gráfico de classificação por streamer
ax=df12[['Hate Speech','Offensive Language','Neither']].plot(figsize=(15,5),kind='bar')
ax.set_xticklabels(df12['Streamer'], rotation=45)
plt.legend(loc='best')
plt.xlabel('Streamers')
plt.ylabel('Quantidade por Classe')
plt.title('Classificação de Hate Speech, Offensive Language e Neither por Streamer')

#Plotando gráfico de classificação de todo o dataset
ax2=df13[['Hate Speech','Offensive Language','Neither']].plot(figsize=(10,5),kind='bar')
ax2.set_xticklabels(df13['All Twitch Dataset'], rotation=0)
plt.legend(loc='best')
plt.xlabel('Comentários de todo o Dataset')
plt.ylabel('Quantidade por Classe')
plt.title('Classificação de Hate Speech,Offensive Language e Neither para todo o Dataset')

#print(" Hate Speech: %d\n" % hs11,
      #"Offenseive Language: %d \n" % ol11,
      #"Neither: %d" % nt11)